In [5]:
import numpy
import pandas
import random
import sklearn
import lightgbm

In [7]:
# 求职者
person = pandas.read_csv("../trainset/person.csv", header=0,
                         names=["求职者编号", "性別", "工作年限", "最高学历", "应聘者专业", "年龄", "最近工作岗位", "最近所在行业", "当前工作所在地", "语言能力",
                                "专业特长"], encoding='gb18030')
person_sex = (person.性別 == "女").astype("float")
person_edu = person.最高学历.map({"其它": 0, "中专": 1, "高中（职高、技校）": 2, "大专": 3, "大学本科": 4, "硕士研究生": 5, "博士研究生": 6, "博士后": 7})
person_major = person.应聘者专业.astype("category")


In [9]:
# 求职意向
intent_table = pandas.read_csv("../trainset/person_cv.csv", header=0,
                               names=["求职者编号", "自荐信", "岗位类别", "工作地点", "所在行业", "可到职天数", "其他说明"])
intent_table["自荐信字数"] = intent_table.自荐信.str.len()

In [10]:
#工作经历
work_history = pandas.read_csv("../trainset/person_job_hist.csv", header=0,
                               names=["求职者编号", "岗位类别", "单位所在地", "单位所属行业", "主要业绩"])
work_history["主要业绩字数"] = work_history.主要业绩.str.len()

In [12]:
# 证书
cert_table = pandas.read_csv("../trainset/person_pro_cert.csv", header=0, names=["求职者编号", "专业证书名称", "备注"])

In [14]:
# 项目经历
project_history = pandas.read_csv("../trainset/person_project.csv", header=0,
                                  names=["求职者编号", "项目名称", "项目说明", "职责说明", "关键技术"])

In [109]:
# 岗位表
job_table = pandas.read_csv("../trainset/recruit.csv", header=0,
                            names=["岗位编号", "招聘对象代码", "招聘对象", "招聘职位", "对应聘者的专业要求", "岗位最低学历", "岗位工作地点", "岗位工作年限", "具体要求"])
job_table.招聘对象代码 = job_table.招聘对象代码.fillna(-1).astype("category")
job_table.招聘对象 = job_table.招聘对象.astype("category")
job_table.岗位最低学历 = job_table.岗位最低学历.map(
    {"其它": 0, "中专": 1, "高中（职高、技校）": 2, "大专": 3, "大学本科": 4, "硕士研究生": 5, "博士研究生": 6, "博士后": 7})
job_table.岗位工作年限 = job_table.岗位工作年限.map({"不限": -1, "应届毕业生": 0, "0至1年": 0, "1至2年": 1, "3至5年": 3, "5年以上": 5})
job_table["具体要求字数"] = job_table.具体要求.str.len()


In [110]:
job_table

,岗位编号,招聘对象代码,招聘对象,招聘职位,对应聘者的专业要求,岗位最低学历,岗位工作地点,岗位工作年限,具体要求,具体要求字数
0,135144,-1.0,NaN,业务员,NaN,2.0,深圳市,0,NaN,NaN
1,137045,-1.0,NaN,电子技术支持工程师,电子信息工程学,1.0,龙岗区,0,要求电子类中专以上文凭，为人诚恳，工作负责，熟悉PROTEL****自动排版系统；对计算机常...,70.0
2,146798,-1.0,NaN,仓管,【工商管理】,1.0,龙岗区,0,电子厂工作经验2年以上（非电子厂勿扰！）；十分熟悉仓库管理流程；懂金碟登帐者优先!,41.0
3,436321,2.0,社会无职,销售代表,NaN,1.0,深圳市,0,女，年龄****-****，高中以上学历，普通话粤语流利，善于沟通，有销售经验者优先。,43.0
4,440725,99.0,不限,造价员,工民建,1.0,深圳市,3,所学专业可以是其他专业，但应有工业与民用建筑土建工程、或给排水工程、或强弱电工程、或通风与空...,109.0
...,...,...,...,...,...,...,...,...,...,...
2099,44263687,1.0,社会在职,跟单文员,NaN,2.0,深圳市,1,女 1名 高中或以上文化，有两年以上丝移印跟单工作经验，字迹端正秀丽，责任心强。,42.0
2100,44274688,3.0,应届毕业,英语播音员,英语,3.0,宝安区,0,男女不限，英语本科以上学历，已过专业英语八级或者普通话二甲，口语好，音质甜美，善于模仿，有播...,66.0
2101,44285651,1.0,社会在职,电气工程师,NaN,NaN,深圳市,5,NaN,NaN
2102,44298679,2.0,社会无职,仓库管理员,NaN,2.0,深圳市,0,1. 1年以上电子厂仓库管理经验2. 熟练金碟登帐3. 能吃苦耐劳。4. 限女性,40.0


In [17]:
work_history_table = work_history.groupby("求职者编号").aggregate({"岗位类别": "count", "主要业绩字数": ["mean", "sum"]}).reset_index()
work_history_table.columns = ["求职者编号", "工作经历数", "平均主要业绩字数", "总主要业绩字数"]
project_history_table = project_history.groupby("求职者编号").aggregate({"项目名称": "count"}).reset_index()
project_history_table.columns = ["求职者编号", "项目经验数"]

In [28]:
work_history

,求职者编号,岗位类别,单位所在地,单位所属行业,主要业绩,主要业绩字数
0,1281276,行政管理,罗湖区,其它,1.协助总经理处理好日常事务及和外部公共关系；2.负责协助起草总经理各类工作往来文件，并负责...,158.0
1,980158,售前/售后服务,NaN,NaN,"本人主要负责万佳,天虹,岁宝,民润等重要客户的品牌分类管理,收集竞争对手信息与反馈,店内执行...",80.0
2,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事学生管理工作.并负责分校区的学生心理辅导和职业指导工作.,32.0
3,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事心理学的教学工作，并担任学校的心理辅导老师，负责了学校心理咨询中心的组建和日常咨...,79.0
4,2980989,产品开发,宝安区,电子行业,"1.工程师对产品进行设计及开发 2.处理3D图和2D图,同时制作相关的资料(如BOM\技术文...",93.0
...,...,...,...,...,...,...
115345,6198806,营销管理,深圳市,建筑工程业,公司从事城市夜景照明泛光亮化方面的工程设****公司主要客户群体为市政工程、房地产开发企业、...,223.0
115346,5863441,运营管理,NaN,信息行业（IT/通讯/互联网）,国内领先的微信开发服务商。基于微信为企业提供开发、运营、培训、推广一体化解决方案，帮助企业实...,563.0
115347,1408390,销售管理,深圳市,服装行业,****公司成****公司的销售，运营，招聘，采购，期****公司的采购和店铺销售和人员管理,46.0
115348,834069,人力资源管理,深圳市,珠宝玉石行业,在职期间任行政总监 及hrbp,15.0


In [32]:
work_history#[work_history.求职者编号==488]

,求职者编号,岗位类别,单位所在地,单位所属行业,主要业绩,主要业绩字数
0,1281276,行政管理,罗湖区,其它,1.协助总经理处理好日常事务及和外部公共关系；2.负责协助起草总经理各类工作往来文件，并负责...,158.0
1,980158,售前/售后服务,NaN,NaN,"本人主要负责万佳,天虹,岁宝,民润等重要客户的品牌分类管理,收集竞争对手信息与反馈,店内执行...",80.0
2,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事学生管理工作.并负责分校区的学生心理辅导和职业指导工作.,32.0
3,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事心理学的教学工作，并担任学校的心理辅导老师，负责了学校心理咨询中心的组建和日常咨...,79.0
4,2980989,产品开发,宝安区,电子行业,"1.工程师对产品进行设计及开发 2.处理3D图和2D图,同时制作相关的资料(如BOM\技术文...",93.0
...,...,...,...,...,...,...
115345,6198806,营销管理,深圳市,建筑工程业,公司从事城市夜景照明泛光亮化方面的工程设****公司主要客户群体为市政工程、房地产开发企业、...,223.0
115346,5863441,运营管理,NaN,信息行业（IT/通讯/互联网）,国内领先的微信开发服务商。基于微信为企业提供开发、运营、培训、推广一体化解决方案，帮助企业实...,563.0
115347,1408390,销售管理,深圳市,服装行业,****公司成****公司的销售，运营，招聘，采购，期****公司的采购和店铺销售和人员管理,46.0
115348,834069,人力资源管理,深圳市,珠宝玉石行业,在职期间任行政总监 及hrbp,15.0


In [38]:
project_history_table[project_history_table.求职者编号==6021388]

,求职者编号,项目经验数
849,6021388,9


In [16]:
# 待匹配标签
train_data = pandas.read_csv("../trainset/recruit_folder.csv", header=0, names=["岗位编号", "求职者编号", "标签"])
test_data = pandas.read_csv("../testset/recruit_folder.csv", header=0, names=["岗位编号", "求职者编号", "标签"])

In [39]:
data = pandas.concat([test_data, train_data], ignore_index=True)
candidate_data = data.groupby("求职者编号").aggregate({"岗位编号": "count"}).reset_index()
candidate_data.columns = ["求职者编号", "求职者数"]
job_data = data.groupby("岗位编号").aggregate({"求职者编号": "count"}).reset_index()
job_data.columns = ["岗位编号", "岗位数"]

In [142]:
def get_data(data, feature):
    candidate_feature = feature.groupby("求职者编号").aggregate({"标签": "mean"}).reset_index()
    candidate_feature.columns = ["求职者编号", "求职者平均标签"]
    job_feature = feature.groupby("岗位编号").aggregate({"标签": "mean"}).reset_index()
    job_feature.columns = ["岗位编号", "岗位平均标签"]

    data = data.merge(person, on="求职者编号", how="left")
    data = data.merge(intent_table, on="求职者编号", how="left")
    data = data.merge(job_table, on="岗位编号", how="left")
    data = data.merge(project_history_table, on="求职者编号", how="left")
    data = data.merge(work_history_table, on="求职者编号", how="left")
    data = data.merge(candidate_data, on="求职者编号", how="left")
    data = data.merge(job_data, on="岗位编号", how="left")
    data = data.merge(candidate_feature, on="求职者编号", how="left")
    data = data.merge(job_feature, on="岗位编号", how="left")
    data["工作地点符合否"] = (data.工作地点 == data.岗位工作地点).astype("float")
#     print(data.columns)
    data_feature = data[["岗位编号", "求职者编号", "标签", "性別", "工作年限", "最高学历", "应聘者专业", "年龄", "自荐信字数", "可到职天数"
        , "项目经验数", "工作经历数", "平均主要业绩字数", "总主要业绩字数", "招聘对象代码", "招聘对象", "岗位最低学历", "岗位工作年限", "具体要求字数", "工作地点符合否"
        , "求职者数", "岗位数", "求职者平均标签", "岗位平均标签"
                             ]]

    data_feature = data_feature[["岗位编号", "求职者编号", "标签"] + [column for column in data_feature.columns if
                                                                  column not in ["岗位编号", "求职者编号", "标签"]]]

    return data_feature

In [143]:
k = 4
train = None
for i in range(k):
    train_label = train_data[train_data.index % k == i].reset_index(drop=True)
    train_feature = train_data[train_data.index % k != i].reset_index(drop=True)

    data_table = get_data(train_label, train_feature)
    train = pandas.concat([train, data_table], ignore_index=True)
test = get_data(test_data, train_data)

In [144]:
train

,岗位编号,求职者编号,标签,性別,工作年限,最高学历,应聘者专业,年龄,自荐信字数,可到职天数,...,招聘对象代码,招聘对象,岗位最低学历,岗位工作年限,具体要求字数,工作地点符合否,求职者数,岗位数,求职者平均标签,岗位平均标签
0,825081,6256839,0,男,6,大学本科,NaN,28,NaN,30.0,...,2.0,社会无职,3.0,3,23.0,0.0,6,93,0.0,0.000000
1,773645,6208645,0,男,0,中专,计算机应用技术,28,NaN,7.0,...,2.0,社会无职,2.0,3,46.0,1.0,1,193,NaN,0.000000
2,820496,5869866,1,女,9,大学本科,电子商务,31,NaN,7.0,...,1.0,社会在职,2.0,0,36.0,0.0,2,39,0.0,0.500000
3,779698,6161445,0,男,0,大专,工程管理,28,114.0,7.0,...,1.0,社会在职,1.0,1,71.0,0.0,1,593,NaN,0.000000
4,42438660,318499951,1,男,0,高中（职高、技校）,NaN,29,97.0,7.0,...,99.0,不限,1.0,0,60.0,0.0,1,33,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35286,786566,5589629,0,男,0,大专,电气工程及其自动化,32,79.0,7.0,...,2.0,社会无职,3.0,1,135.0,0.0,15,85,0.0,0.000000
35287,762700,6057161,0,男,8,大专,物流管理,31,135.0,7.0,...,-1.0,NaN,2.0,0,NaN,0.0,2,388,NaN,0.000000
35288,800235,6151744,0,男,7,大学本科,平面设计,30,440.0,7.0,...,-1.0,NaN,3.0,0,255.0,1.0,3,34,0.0,0.000000
35289,811206,6262045,0,女,0,高中（职高、技校）,办公自动化,30,NaN,7.0,...,99.0,不限,NaN,0,20.0,1.0,1,109,NaN,0.041667


In [149]:
data= pandas.concat([train, test], ignore_index=True)

In [150]:
data_feature = pd.get_dummies(data[['性別','工作年限','最高学历','应聘者专业','可到职天数','招聘对象代码', '招聘对象',
       '岗位最低学历','岗位工作年限','工作地点符合否', '求职者数', '岗位数', '求职者平均标签','岗位平均标签']])#[['性别']]#[']性别']]
data_feature['年龄']=data['年龄']
data_feature['项目经验数']=data['项目经验数']
data_feature['具体要求字数']=data['具体要求字数']

In [155]:
data_feature

,工作年限,可到职天数,岗位最低学历,岗位工作年限,工作地点符合否,求职者数,岗位数,求职者平均标签,岗位平均标签,性別_女,...,招聘对象代码_2.0,招聘对象代码_3.0,招聘对象代码_99.0,招聘对象_不限,招聘对象_应届毕业,招聘对象_社会在职,招聘对象_社会无职,年龄,项目经验数,具体要求字数
0,6,30.0,3.0,3,0.0,6,93,0.0,0.0,0,...,1,0,0,0,0,0,1,28,NaN,23.0
1,0,7.0,2.0,3,1.0,1,193,NaN,0.0,0,...,1,0,0,0,0,0,1,28,NaN,46.0
2,9,7.0,2.0,0,0.0,2,39,0.0,0.5,1,...,0,0,0,0,0,1,0,31,NaN,36.0
3,0,7.0,1.0,1,0.0,1,593,NaN,0.0,0,...,0,0,0,0,0,1,0,28,NaN,71.0
4,0,7.0,1.0,0,0.0,1,33,NaN,1.0,0,...,0,0,1,1,0,0,0,29,NaN,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106060,5,7.0,3.0,5,1.0,1,9,NaN,1.0,0,...,0,0,1,1,0,0,0,26,1.0,79.0
106061,0,7.0,3.0,5,1.0,1,9,NaN,1.0,0,...,0,0,1,1,0,0,0,28,NaN,79.0
106062,0,7.0,NaN,5,0.0,1,1,NaN,NaN,1,...,0,0,0,0,0,1,0,28,NaN,203.0
106063,7,7.0,NaN,5,0.0,1,3,NaN,1.0,0,...,0,1,0,0,1,0,0,29,NaN,56.0


In [173]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

train_feature,test_feature,train_label,test_label =train_test_split(data_feature[:train.shape[0]].values,train["标签"])
train_set=lightgbm.Dataset(train_feature, label=train_label)
val_set=lightgbm.Dataset(test_feature, label=test_label)

model = lightgbm.train(train_set=train_set,
                       valid_sets=[val_set],
                       num_boost_round=1000,
                       
                       params={"objective": "binary", "learning_rate": 0.03, "max_depth": 6, "num_leaves": 32,
                               "verbose": -1, "bagging_fraction": 0.8, "feature_fraction": 0.8},
                       feval=lgb_f1_score,
                      early_stopping_rounds=100)


[1]	valid_0's binary_logloss: 0.427244	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.407921	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.391078	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.382788	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.368711	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.355838	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.344192	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.338504	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.328141	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.318525	valid_0's f1: 0
[11]	valid_0's binary_logloss: 0.309583	valid_0's f1: 0
[12]	valid_0's binary_logloss: 0.301238	valid_0's f1: 0
[13]	valid_0's binary_logloss: 0.293438	valid_0's f1: 0
[14]	valid_0's binary_logloss: 0.289755	valid_0's f1: 0
[15]	valid_0's binary_logloss: 0.28276	valid_0's f1: 0
[16]	valid_0's binary_logloss: 0.279229	valid_0's f1: 0
[17]	valid_0's binary_logloss: 0.272672	valid_0's f1

[138]	valid_0's binary_logloss: 0.126158	valid_0's f1: 0.849707
[139]	valid_0's binary_logloss: 0.126007	valid_0's f1: 0.849395
[140]	valid_0's binary_logloss: 0.125902	valid_0's f1: 0.849395
[141]	valid_0's binary_logloss: 0.12576	valid_0's f1: 0.849084
[142]	valid_0's binary_logloss: 0.125654	valid_0's f1: 0.849084
[143]	valid_0's binary_logloss: 0.125558	valid_0's f1: 0.849084
[144]	valid_0's binary_logloss: 0.125436	valid_0's f1: 0.848773
[145]	valid_0's binary_logloss: 0.125354	valid_0's f1: 0.848773
[146]	valid_0's binary_logloss: 0.125284	valid_0's f1: 0.848773
[147]	valid_0's binary_logloss: 0.125207	valid_0's f1: 0.848773
[148]	valid_0's binary_logloss: 0.125086	valid_0's f1: 0.848773
[149]	valid_0's binary_logloss: 0.124986	valid_0's f1: 0.848773
[150]	valid_0's binary_logloss: 0.124878	valid_0's f1: 0.848773
[151]	valid_0's binary_logloss: 0.124811	valid_0's f1: 0.849195
[152]	valid_0's binary_logloss: 0.124751	valid_0's f1: 0.849084
[153]	valid_0's binary_logloss: 0.12469	v

[274]	valid_0's binary_logloss: 0.121623	valid_0's f1: 0.853614
[275]	valid_0's binary_logloss: 0.121611	valid_0's f1: 0.853614
[276]	valid_0's binary_logloss: 0.121606	valid_0's f1: 0.853614
[277]	valid_0's binary_logloss: 0.121615	valid_0's f1: 0.853198
[278]	valid_0's binary_logloss: 0.121624	valid_0's f1: 0.852781
[279]	valid_0's binary_logloss: 0.121612	valid_0's f1: 0.852781
[280]	valid_0's binary_logloss: 0.121596	valid_0's f1: 0.852471
[281]	valid_0's binary_logloss: 0.121609	valid_0's f1: 0.852471
[282]	valid_0's binary_logloss: 0.121603	valid_0's f1: 0.852888
[283]	valid_0's binary_logloss: 0.121608	valid_0's f1: 0.852888
[284]	valid_0's binary_logloss: 0.121569	valid_0's f1: 0.853508
[285]	valid_0's binary_logloss: 0.121565	valid_0's f1: 0.853614
[286]	valid_0's binary_logloss: 0.121563	valid_0's f1: 0.853924
[287]	valid_0's binary_logloss: 0.121576	valid_0's f1: 0.853924
[288]	valid_0's binary_logloss: 0.121582	valid_0's f1: 0.853508
[289]	valid_0's binary_logloss: 0.121568

[403]	valid_0's binary_logloss: 0.121259	valid_0's f1: 0.854453
[404]	valid_0's binary_logloss: 0.121266	valid_0's f1: 0.854038
[405]	valid_0's binary_logloss: 0.121276	valid_0's f1: 0.854038
[406]	valid_0's binary_logloss: 0.121269	valid_0's f1: 0.854038
[407]	valid_0's binary_logloss: 0.121281	valid_0's f1: 0.853623
[408]	valid_0's binary_logloss: 0.121283	valid_0's f1: 0.854038
[409]	valid_0's binary_logloss: 0.121286	valid_0's f1: 0.854038
[410]	valid_0's binary_logloss: 0.121298	valid_0's f1: 0.854038
[411]	valid_0's binary_logloss: 0.121299	valid_0's f1: 0.854038
[412]	valid_0's binary_logloss: 0.121296	valid_0's f1: 0.854038
[413]	valid_0's binary_logloss: 0.121283	valid_0's f1: 0.854038
[414]	valid_0's binary_logloss: 0.121297	valid_0's f1: 0.854038
[415]	valid_0's binary_logloss: 0.12132	valid_0's f1: 0.853623
[416]	valid_0's binary_logloss: 0.121305	valid_0's f1: 0.853623
[417]	valid_0's binary_logloss: 0.121308	valid_0's f1: 0.853314
[418]	valid_0's binary_logloss: 0.121312	

In [158]:
data_feature.values.shape

(106065, 450)

In [154]:
train.标签.value_counts()

0    29670
1     5621
Name: 标签, dtype: int64

In [152]:
test["预测打分"] = model.predict(data_feature.values[train.shape[0]:])
test = test.sort_values("预测打分", ascending=False, ignore_index=True)
test["预测"] = 0
test.loc[:int(0.15 * len(test)), ["预测"]] = 1

submit = test.loc[:, ["岗位编号", "求职者编号", "预测"]]
submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
submit.to_csv("result.csv", index=False)

In [102]:
train.columns

Index(['岗位编号', '求职者编号', '标签', '性別', '工作年限', '最高学历', '应聘者专业', '年龄', '自荐信字数',
       '可到职天数', '项目经验数', '工作经历数', '平均主要业绩字数', '总主要业绩字数', '招聘对象代码', '招聘对象',
       '岗位最低学历', '岗位工作年限', '具体要求字数', '工作地点符合否', '求职者数', '岗位数', '求职者平均标签',
       '岗位平均标签'],
      dtype='object')

In [106]:
train[["性別"]]

,性別
0,男
1,男
2,女
3,男
4,男
...,...
35286,男
35287,男
35288,男
35289,女
